# Building Models in PyTorch


## `torch.nn.Module` and `torch.nn.Parameter`

In this video, we'll be discussing some of the tools PyTorch makes available for building deep learning networks.

Except for `Parameter`, the classes we discuss in this video are all subclasses of `torch.nn.Module`. This is the PyTorch base class meant to encapsulate behaviors specific to PyTorch Models and their components.

One important behavior of `torch.nn.Module` is registering parameters. If a particular `Module` subclass has learning weights, these weights are expressed as instances of `torch.nn.Parameter`. The `Parameter` class is a subclass of `torch.Tensor`, with the special behavior that when they are assigned as attributes of a `Module`, they are added to the list of that modules parameters. These parameters may be accessed through the `parameters()` method on the `Module` class.

As a simple example, here's a very simple model with two linear layers and an activation function. We'll create an instance of it and ask it to report on its parameters:

In [1]:
import torch

class TinayModel(torch.nn.Module):
    def __init__(self):
        super(TinayModel, self).__init__()

        self.linear1 = torch.nn.Linear(100,200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x
    
tinaymodel = TinayModel()

print("The model:")
print(tinaymodel)

print('\n\n just one layer')
print(tinaymodel.linear2)

print('\n\n model params')
for param in tinaymodel.parameters():
    print(param)

print('\n\n layer param')
for param in tinaymodel.linear2.parameters():
    print(param)

The model:
TinayModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


 just one layer
Linear(in_features=200, out_features=10, bias=True)


 model params
Parameter containing:
tensor([[ 0.0852,  0.0909, -0.0529,  ..., -0.0499,  0.0445, -0.0573],
        [ 0.0998,  0.0379, -0.0385,  ..., -0.0900, -0.0885, -0.0976],
        [-0.0292, -0.0019,  0.0965,  ...,  0.0540,  0.0687,  0.0552],
        ...,
        [ 0.0400,  0.0113, -0.0117,  ...,  0.0125,  0.0899, -0.0725],
        [ 0.0230, -0.0195, -0.0730,  ..., -0.0736,  0.0498, -0.0094],
        [-0.0283,  0.0353, -0.0750,  ...,  0.0107, -0.0722,  0.0340]],
       requires_grad=True)
Parameter containing:
tensor([-1.6741e-02,  8.3074e-02, -7.4763e-02, -7.8155e-02,  7.5941e-02,
        -2.6809e-02, -4.1605e-02,  2.5870e-02,  7.2150e-02, -2.4994e-02,
         6.1980e-02,  1.6762e-02,  4.3838e-02,  7.658

This shows the fundamental structure of a PyTorch model: there is an `__init__()` method that defines the layers and other components of a model, and a `forward()` method where the computation gets done. Note that we can print the model, or any of its submodules, to learn about its structure.

## Common Layer Types

### Linear Layers

The most basic type of neural network layer is a *linear* or *fully connected* layer. This is a layer where every input influences every output of the layer to a degree specified by the layer's weights. If a model has *m* inputs and *n* outputs, the weights will be an *m * n* matrix. For example:

In [2]:
lin  = torch.nn.Linear(3,2)
x = torch.rand(1, 3)
print('Input')
print(x)

print('\n\nWeight and Bias parameters')
for param in lin.parameters():
    print(param)

y = lin(x)
print('\n\noutput')
print(y)


Input
tensor([[0.8622, 0.2884, 0.1907]])


Weight and Bias parameters
Parameter containing:
tensor([[ 0.5000,  0.5531,  0.4281],
        [-0.3970,  0.3367,  0.2970]], requires_grad=True)
Parameter containing:
tensor([ 0.0600, -0.1127], requires_grad=True)


output
tensor([[ 0.7323, -0.3013]], grad_fn=<AddmmBackward0>)


If you do the matrix multiplication of `x` by the linear layer's weights, and add the biases, you'll find that you get the output vector `y`.

One other important feature to note: When we checked the weights of our layer with `lin.weight`, it reported itself as a `Parameter` (which is a subclass of `Tensor`), and let us know that it's tracking gradients with autograd. This is a default behavior for `Parameter` that differs from `Tensor`.

Linear layers are used widely in deep learning models. One of the most common places you'll see them is in classifier models, which will usually have one or more linear layers at the end, where the last layer will have *n* outputs, where *n* is the number of classes the classifier addresses.

### Convolutional Layers

*Convolutional* layers are built to handle data with a high degree of spatial correlation. They are very commonly used in computer vision, where they detect close groupings of features which the compose into higher-level features. They pop up in other contexts too - for example, in NLP applications, where the a word's immediate context (that is, the other words nearby in the sequence) can affect the meaning of a sentence.

We saw convolutional layers in action in LeNet5 in an earlier video:

In [3]:
import torch.functional as F
class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet).__init__()
        # 1 input image channel (black & white), 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # an affine operation: y = wx + b
        self.fc1 = torch.nn.Linear(16, 6, 6, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # Max poling over (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # if the size is square you can only specify a single number only
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:] #all dimention except the batch dimention
        num_feature = 1
        for s in size:
            num_feature *= s 
        return num_feature

Let's break down what's happening in the convolutional layers of this model. Starting with `conv1`:

* LeNet5 is meant to take in a 1x32x32 black & white image. **The first argument to a convolutional layer's constructor is the number of input channels.** Here, it is 1. If we were building this model to look at 3-color channels, it would be 3.
* A convolutional layer is like a window that scans over the image, looking for a pattern it recognizes. These patterns are called *features,* and one of the parameters of a convolutional layer is the number of features we would like it to learn. **This is the second argument to the constructor is the number of output features.** Here, we're asking our layer to learn 6 features.
* Just above, I likened the convolutional layer to a window - but how big is the window? **The third argument is the window or *kernel* size.** Here, the "5" means we've chosen a 5x5 kernel. (If you want a kernel with height different from width, you can specify a tuple for this argument - e.g., `(3, 5)` to get a 3x5 convolution kernel.)

The output of a convolutional layer is an *activation map* - a spatial representation of the presence of features in the input tensor. `conv1` will give us an output tensor of 6x28x28; 6 is the number of features, and 28 is the height and width of our map. (The 28 comes from the fact that when scanning a 5-pixel window over a 32-pixel row, there are only 28 valid positions.)

We then pass the output of the convolution through a ReLU activation function (more on activation functions later), then through a max pooling layer. The max pooling layer takes features near each other in the activation map and groups them together. It does this by reducing the tensor, merging every 2x2 group of cells in the output into a single cell, and assigning that cell the maximum value of the 4 cells that went into it. This gives us a lower-resolution version of the activation map, with dimensions 6x14x14.

Our next convolutional layer, `conv2`, expects 6 input channels (corresponding to the 6 features sought by the first layer), has 16 output channels, and a 3x3 kernel. It puts out a 16x12x12 activation map, which is again reduced by a max pooling layer to 16x6x6. Prior to passing this output to the linear layers, it is reshaped to a 16 * 6 * 6 = 576-element vector for consumption by the next layer.

There are convolutional layers for addressing 1D, 2D, and 3D tensors. There are also many more optional arguments for a conv layer constructor, including stride length(e.g., only scanning every second or every third position) in the input, padding (so you can scan out to the edges of the input), and more. See the [documentation](https://pytorch.org/docs/stable/nn.html#convolution-layers) for more information.

### Recurrent Layers

*Recurrent neural networks* (or *RNNs)* are used for sequential data - anything from time-series measurements from a scientific instrument to natural language sentences to DNA nucleotides. An RNN does this by maintaining a *hidden state* that acts as a sort of memory for what it has seen in the sequence so far.

The internal structure of an RNN layer - or its variants, the LSTM (long short-term memory) and GRU (gated recurrent unit) - is moderately complex and beyond the scope of this video, but we'll show you what one looks like in action with an LSTM-based part-of-speech tagger (a type of classifier that tells you if a word is a noun, verb, etc.):

In [4]:
class LSTMTagger(torch.nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(self, LSTMTagger).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)
        
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)
        
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
        